# CHAPTER 7 - VQE: Variational Quantum Eigensolver - PennyLane Code

*Note*: You may skip the following three cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.

In [ ]:
pip install pennylane==0.26.0


In [ ]:
pip install pennylane_qiskit==0.24.0

In [ ]:
import pennylane as qml
from pennylane import numpy as np

seed = 1234
np.random.seed(seed) 

symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.6991986158, 0.0, 0.0, 0.6991986158])

H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Qubit Hamiltonian: ")
print(H)

In [ ]:
nqubits = 4
def EfficientSU2(theta):
    for i in range(nqubits):
      qml.RY(theta[i], wires = i)
      qml.RZ(theta[i+nqubits], wires = i)
    for i in range(nqubits-1):
      qml.CNOT(wires = [i, i + 1])
    for i in range(nqubits):
      qml.RY(theta[i+2*nqubits], wires = i)
      qml.RZ(theta[i+3*nqubits], wires = i)

In [ ]:
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def energy(param):
    EfficientSU2(param)
    return qml.expval(H)

In [ ]:
from scipy.optimize import minimize

theta = np.array(np.random.random(4*nqubits), requires_grad=True)
result = minimize(energy, x0=theta)

print("Optimal parameters", result.x)
print("Energy", result.fun)

*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [ ]:
from qiskit import IBMQ

ibm_token="1234"


In [ ]:
from pennylane_qiskit import upload_vqe_runner, vqe_runner

IBMQ.enable_account(ibm_token)

program_id = upload_vqe_runner(hub="ibm-q", group="open", project="main")

job = vqe_runner(
    program_id=program_id,
    backend="ibm_oslo",
    hamiltonian=H,
    ansatz=EfficientSU2,
    x0=np.array(np.random.random(4*nqubits)),
    shots=1024,
    optimizer="SPSA",
    kwargs={"hub": "ibm-q", "group": "open", "project": "main"}
)


In [ ]:
print(job.result())

In [ ]:
from pennylane_qiskit import upload_vqe_runner, vqe_runner

program_id = upload_vqe_runner(hub="ibm-q", group="open", project="main")

job = vqe_runner(
    program_id=program_id,
    backend="ibm_oslo",
    hamiltonian=H,
    ansatz=EfficientSU2,
    x0=np.array(np.random.random(4*nqubits)),
    shots=1024,
    optimizer="SPSA",
    kwargs={"hub": "ibm-q", "group": "open", "project": "main"},
    use_measurement_mitigation = True
)

print(job.result())
